In [1]:
import pandas as pd
import spotipy
from fycharts.SpotifyCharts import SpotifyCharts
from spotipy.oauth2 import SpotifyClientCredentials

In [50]:
df = pd.read_csv('regional-global-weekly-2017-08-04--2017-08-11.csv')

In [51]:
df

,Unnamed: 0,Unnamed: 1,Unnamed: 2,Note that these figures are generated using a formula that protects against any artificial inflation of chart positions.,Unnamed: 4
0,Position,Track Name,Artist,Streams,URL
1,1,Mi Gente,J Balvin,32361062,https://open.spotify.com/track/2rb5MvYT7ZIxbKW...
2,2,Despacito - Remix,Luis Fonsi,28238094,https://open.spotify.com/track/5CtI0qwDJkDQGwX...
3,3,Wild Thoughts (feat. Rihanna & Bryson Tiller),DJ Khaled,25365011,https://open.spotify.com/track/1OAh8uOEOvTDqkK...
4,4,Unforgettable,French Montana,23957823,https://open.spotify.com/track/3B54sVLJ402zGa6...
...,...,...,...,...,...
196,196,Vente Pa' Ca (feat. Maluma),Ricky Martin,3003473,https://open.spotify.com/track/7DM4BPaS7uofFul...
197,197,ELEMENT.,Kendrick Lamar,2987571,https://open.spotify.com/track/1EaKU4dMbesXXd3...
198,198,Wonderwall - Remastered,Oasis,2975831,https://open.spotify.com/track/5qqabIl2vWzo9Ap...
199,199,Ahora Me Llama,KAROL G,2971762,https://open.spotify.com/track/61HHDBLqF3AmSvL...


In [52]:
df.columns = df.iloc[0]

In [53]:
df = df.iloc[1:].reset_index()

In [54]:
df.URL.astype('str')

0      https://open.spotify.com/track/2rb5MvYT7ZIxbKW...
1      https://open.spotify.com/track/5CtI0qwDJkDQGwX...
2      https://open.spotify.com/track/1OAh8uOEOvTDqkK...
3      https://open.spotify.com/track/3B54sVLJ402zGa6...
4      https://open.spotify.com/track/5bcTCxgc7xVfSaM...
                             ...                        
195    https://open.spotify.com/track/7DM4BPaS7uofFul...
196    https://open.spotify.com/track/1EaKU4dMbesXXd3...
197    https://open.spotify.com/track/5qqabIl2vWzo9Ap...
198    https://open.spotify.com/track/61HHDBLqF3AmSvL...
199    https://open.spotify.com/track/4tCtwWceOPWzenK...
Name: URL, Length: 200, dtype: object

In [55]:
df.URL = df.URL.apply(lambda x : x[31:])

In [56]:
df.rename(columns={'URL':'ID'}, inplace=True)

In [57]:
credentials = SpotifyClientCredentials(client_id="6b8119b6cec94f538151d6bb7fd99acd", client_secret="bb3cc9f8fdb04ddb9516ab01c7bfc257")
token = credentials.get_access_token()

spotify = spotipy.Spotify(auth=token)

features = spotify.audio_features(str(df['ID'].iloc[0]))


/Users/Antonio/OneDrive - Politecnico di Milano/Università_Magistrale/Materiale_Corsi/II_Anno/Data_Viz/env/lib/python3.7/site-packages/ipykernel_launcher.py:2: DeprecationWarning: You're using 'as_dict = True'.get_access_token will return the token string directly in future versions. Please adjust your code accordingly, or use get_cached_token instead.
  


In [58]:
features

[{'danceability': 0.543,
  'energy': 0.677,
  'key': 11,
  'loudness': -4.915,
  'mode': 0,
  'speechiness': 0.0993,
  'acousticness': 0.0148,
  'instrumentalness': 6.21e-06,
  'liveness': 0.13,
  'valence': 0.294,
  'tempo': 103.809,
  'type': 'audio_features',
  'id': '2rb5MvYT7ZIxbKW5hfcHx8',
  'uri': 'spotify:track:2rb5MvYT7ZIxbKW5hfcHx8',
  'track_href': 'https://api.spotify.com/v1/tracks/2rb5MvYT7ZIxbKW5hfcHx8',
  'analysis_url': 'https://api.spotify.com/v1/audio-analysis/2rb5MvYT7ZIxbKW5hfcHx8',
  'duration_ms': 189440,
  'time_signature': 4}]

In [59]:
def extract_features(df, n_tracks = 50):
    dataframe = pd.DataFrame(columns=['id','danceability','energy','key','loudness','mode','speechiness',
                                     'acousticness','instrumentalness','liveness','valence','tempo','time_signature',
                                     'duration_ms'])
    for i in range(n_tracks):
        features = spotify.audio_features(str(df['ID'].iloc[i]))
        print(features[0]['danceability'])
        dataframe.append({'danceability' : features[0]['danceability'],
        'energy': features[0]['energy'],
        'key' : features[0]['key'],
        'loudness' : features[0]['loudness'],
        'mode' : features[0]['mode'],
        'speechiness' : features[0]['speechiness'],
        'acousticness' : features[0]['acousticness'],
        'instrumentalness' : features[0]['instrumentalness'],
        'liveness' : features[0]['liveness'],
        'valence' : features[0]['valence'],
        'tempo' : features[0]['tempo'],
        'time_signature' : features[0]['time_signature'],
        'duration_ms' : features[0]['duration_ms']}, ignore_index=True)
    return dataframe

In [60]:
def extract_features(df, n_tracks = 50):
    dataframe = pd.DataFrame(columns=['id','danceability','energy','key','loudness','mode','speechiness',
                                     'acousticness','instrumentalness','liveness','valence','tempo','time_signature',
                                     'duration_ms'])
    for i in range(n_tracks):
        features = spotify.audio_features(str(df['ID'].iloc[i]))
        dataframe.loc[i] = [ features[0]['id'],
        features[0]['danceability'],
        features[0]['energy'],
        features[0]['key'],
        features[0]['loudness'],
        features[0]['mode'],
        features[0]['speechiness'],
        features[0]['acousticness'],
        features[0]['instrumentalness'],
        features[0]['liveness'],
        features[0]['valence'],
        features[0]['tempo'],
        features[0]['time_signature'],
        features[0]['duration_ms']]
    return dataframe

In [61]:
prova = extract_features(df, n_tracks=200)

In [62]:
prova

,id,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature,duration_ms
0,2rb5MvYT7ZIxbKW5hfcHx8,0.543,0.677,11,-4.915,0,0.0993,0.014800,0.000006,0.1300,0.294,103.809,4,189440
1,5CtI0qwDJkDQGwXD1H1cLb,0.694,0.815,2,-4.328,1,0.1200,0.229000,0.000000,0.0924,0.813,88.931,4,228827
2,1OAh8uOEOvTDqkKFsKksCi,0.671,0.672,0,-3.094,0,0.0688,0.032900,0.000000,0.1180,0.632,97.979,4,204173
3,3B54sVLJ402zGa6Xm4YGNe,0.726,0.769,6,-5.043,1,0.1230,0.029300,0.010100,0.1040,0.733,97.985,4,233902
4,5bcTCxgc7xVfSaMV3RuVke,0.893,0.745,11,-3.105,0,0.0571,0.064200,0.000000,0.0943,0.872,101.018,4,223413
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,7DM4BPaS7uofFul3ywMe46,0.663,0.920,11,-4.070,0,0.2260,0.004310,0.000017,0.1010,0.533,99.935,4,259196
196,1EaKU4dMbesXXd3BrLCtYG,0.748,0.705,4,-4.547,0,0.4850,0.204000,0.000000,0.2460,0.483,189.891,4,208733
197,5qqabIl2vWzo9ApSC317sa,0.378,0.896,2,-4.066,1,0.0407,0.000452,0.000000,0.1940,0.469,174.302,4,258773
198,61HHDBLqF3AmSvLfHKNGd2,0.664,0.576,7,-5.395,0,0.0349,0.007300,0.000063,0.0912,0.121,134.934,4,233293


In [63]:
output = df.join(prova.set_index('id'), on='ID')

In [64]:
output.to_csv('global_08-2017.csv')

In [65]:
output

,index,Position,Track Name,Artist,Streams,ID,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature,duration_ms
0,1,1,Mi Gente,J Balvin,32361062,2rb5MvYT7ZIxbKW5hfcHx8,0.543,0.677,11,-4.915,0,0.0993,0.014800,0.000006,0.1300,0.294,103.809,4,189440
1,2,2,Despacito - Remix,Luis Fonsi,28238094,5CtI0qwDJkDQGwXD1H1cLb,0.694,0.815,2,-4.328,1,0.1200,0.229000,0.000000,0.0924,0.813,88.931,4,228827
2,3,3,Wild Thoughts (feat. Rihanna & Bryson Tiller),DJ Khaled,25365011,1OAh8uOEOvTDqkKFsKksCi,0.671,0.672,0,-3.094,0,0.0688,0.032900,0.000000,0.1180,0.632,97.979,4,204173
3,4,4,Unforgettable,French Montana,23957823,3B54sVLJ402zGa6Xm4YGNe,0.726,0.769,6,-5.043,1,0.1230,0.029300,0.010100,0.1040,0.733,97.985,4,233902
4,5,5,"Feels (feat. Pharrell Williams, Katy Perry & B...",Calvin Harris,23208440,5bcTCxgc7xVfSaMV3RuVke,0.893,0.745,11,-3.105,0,0.0571,0.064200,0.000000,0.0943,0.872,101.018,4,223413
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,196,196,Vente Pa' Ca (feat. Maluma),Ricky Martin,3003473,7DM4BPaS7uofFul3ywMe46,0.663,0.920,11,-4.070,0,0.2260,0.004310,0.000017,0.1010,0.533,99.935,4,259196
196,197,197,ELEMENT.,Kendrick Lamar,2987571,1EaKU4dMbesXXd3BrLCtYG,0.748,0.705,4,-4.547,0,0.4850,0.204000,0.000000,0.2460,0.483,189.891,4,208733
197,198,198,Wonderwall - Remastered,Oasis,2975831,5qqabIl2vWzo9ApSC317sa,0.378,0.896,2,-4.066,1,0.0407,0.000452,0.000000,0.1940,0.469,174.302,4,258773
198,199,199,Ahora Me Llama,KAROL G,2971762,61HHDBLqF3AmSvLfHKNGd2,0.664,0.576,7,-5.395,0,0.0349,0.007300,0.000063,0.0912,0.121,134.934,4,233293
